## Code pour prendre des revues de différents produits
* Importer les données
    * Appliances
    * Automotive
    * CDs_and_Vinyl
    * Digital_Music
    * Gift_Cards
    * Handmade_Products
    * Musical_Instruments
    * Video_Games
* Filtrer celles que je souhaite conserver (quantité, équilibre des notes, présence d'informations pour xgboost [ex : prix, ...])
    * Ne pas oublier le set.seed
* Enregistrer les données

## Pipeline de données


In [1]:
### Packages de base
import numpy as np
import pandas as pd
from datasets import load_dataset

### CDs and Vinyl
#### Importation

In [2]:
## Load User Reviews
dataset_reviews = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_CDs_and_Vinyl", split="full", trust_remote_code=True)

## Load Item Metadata - Test avec All_Beauty
dataset_items = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_CDs_and_Vinyl", split="full", trust_remote_code=True)


CDs_and_Vinyl.jsonl:   0%|          | 0.00/3.29G [00:00<?, ?B/s]

C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\natha\.cache\huggingface\hub\datasets--McAuley-Lab--Amazon-Reviews-2023. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Generating full split: 0 examples [00:00, ? examples/s]

meta_CDs_and_Vinyl.jsonl:   0%|          | 0.00/949M [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

#### Reviews

In [3]:
## Convert to df (plus à l'aise pour certaines manip et EDA)
df_reviews = pd.DataFrame(dataset_reviews)

In [11]:
print(df_reviews.count())

rating               4827273
title                4827273
text                 4827273
images               4827273
asin                 4827273
parent_asin          4827273
user_id              4827273
timestamp            4827273
helpful_vote         4827273
verified_purchase    4827273
full_text            4827273
as_image             4827273
as_helpful_vote      4827273
dtype: int64


In [5]:
### Data manipulation
## Concat title and text
df_reviews['full_text'] = "Title : " + df_reviews['title'].astype(str) + "\n Review : " + df_reviews['text'].astype(str)

## Create variable as_image
df_reviews['as_image'] = np.where(df_reviews['images'].str.len() == 0, 0, 1)

## Create variable as_helpful_vote
df_reviews['as_helpful_vote'] = np.where(df_reviews['helpful_vote'] == 0, 0, 1)

#### Items

In [4]:
## Convert to pandas df
df_items = pd.DataFrame(dataset_items)

In [7]:
## Main category (filtrer pour conserver seulement "Appliances")
print(df_items['main_category'].value_counts())
df_items_cat = df_items[df_items['main_category'] == "Digital Music"]
print(df_items_cat['main_category'].value_counts())

Digital Music                690764
Movies & TV                    9049
Books                           770
Tools & Home Improvement         88
Amazon Home                      78
Cell Phones & Accessories        51
Musical Instruments              42
All Electronics                  42
Health & Personal Care           39
Toys & Games                     35
Video Games                      25
Sports & Outdoors                21
Software                         21
Office Products                  20
All Beauty                       15
Industrial & Scientific          13
Home Audio & Theater             11
Grocery                          10
Arts, Crafts & Sewing            10
Baby                              4
Computers                         4
Audible Audiobooks                4
Pet Supplies                      4
AMAZON FASHION                    3
Camera & Photo                    2
Collectible Coins                 2
Appliances                        1
Entertainment               

In [8]:
## Filtrer pour conserver seulement les produits avec un prix
print(df_items_cat['price'].count())
print(df_items_cat[df_items_cat['price'] != 'None']['price'].count())
df_items_price = df_items_cat[df_items_cat['price'] != 'None']
df_items_price['price'].astype(float).describe()

690764
546175


count    546175.000000
mean         29.287919
std          49.383735
min           0.010000
25%          10.730000
50%          16.980000
75%          29.980000
max        2949.450000
Name: price, dtype: float64

In [9]:
## Aperçu des catégories
print(df_items_price['categories'].value_counts())


[CDs & Vinyl, Pop]                                                             43741
[CDs & Vinyl, Jazz]                                                            14899
[CDs & Vinyl, Rock]                                                            13484
[CDs & Vinyl, Classical, Chamber Music]                                        11956
[CDs & Vinyl, Classical]                                                       11268
                                                                               ...  
[CDs & Vinyl, Customized Children's Music, All]                                    1
[CDs & Vinyl, Today's Deals in Music, Opera & Vocal, Opera & Vocal General]        1
[CDs & Vinyl, Hannibal Records]                                                    1
[CDs & Vinyl, Today's Deals in Music, R&B, R&B General]                            1
[CDs & Vinyl, Today's Deals in Music, Miscellaneous, Instructional]                1
Name: categories, Length: 835, dtype: int64


In [10]:
## Isoler les catégories importantes
#df_items_price['categories_single'] = df_items_price['categories'].apply(lambda x : (x or [None])[-1])
df_items_price['categories_single'] = df_items_price['categories'].apply(lambda x : (x or [None, None])[1])

## Aperçu des options
print(df_items_price['categories_single'].value_counts())

## À grouper
# Parts & Accessories
# Other


Pop                                                            83921
Classical                                                      56828
International Music                                            44337
Jazz                                                           41587
Indie & Alternative                                            39240
                                                               ...  
Music Coupons                                                      1
Hannibal Records                                                   1
Amazon Exclusives in CDs & Vinyl                                   1
TurboTax Refund Bonus Exclusive: $20 off Select CD Box Sets        1
2014 Best of the Year So Far- CDs & Vinyl                          1
Name: categories_single, Length: 72, dtype: int64


C:\Users\natha\AppData\Local\Temp\ipykernel_20948\3939727900.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items_price['categories_single'] = df_items_price['categories'].apply(lambda x : (x or [None, None])[1])


In [20]:
#value_counts = df_items_price['categories_single'].value_counts()
#values_to_keep = value_counts[value_counts > 10000].index
df_items_price['categories_single'][df_items_price['categories_single'].isin(values_to_keep)].value_counts()

Pop                       83921
Classical                 56828
International Music       44337
Jazz                      41587
Indie & Alternative       39240
Rock                      38388
Today's Deals in Music    26980
Country                   20173
Dance & Electronic        20060
Christian & Gospel        15946
Metal                     14863
Blues                     13843
Classic Rock              12901
R&B                       12756
Folk                      10796
Vinyl Store               10627
Name: categories_single, dtype: int64

In [21]:
## Grouper les catégories
df_items_price['categories_grp'] = np.where(
    np.isin(df_items_price['categories_single'], ["Pop", "Dance & Electronic"]), "Pop", 
    np.where(
        np.isin(df_items_price['categories_single'], ["Classical", "Jazz", "Blues"]), "Classical",
        np.where(
            np.isin(df_items_price['categories_single'], ["Rock", "Metal", "Classic Rock"]), "Rock",
        np.where(
            np.isin(df_items_price['categories_single'], ["Country", "Folk", "Indie & Alternative"]), "Country", "Other"
        ))))

df_items_price['categories_grp'].value_counts()

C:\Users\natha\AppData\Local\Temp\ipykernel_20948\1758998326.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items_price['categories_grp'] = np.where(


Other        193575
Classical    112258
Pop          103981
Country       70209
Rock          66152
Name: categories_grp, dtype: int64

In [11]:
### Potentiel pour XGBoost
## Reviews
# Création variable presence_image (as_image)
# verified_purchase
# helpful_vote (as_helpful_votes)

## Items
# Sure : main_category, average_rating, rating_number, price
# catgories_grp
# Potentiel : 
#   XTitle? De quoi à faire avec ça?
#   *store name? dequoi à faire avec ça?
#   **categories (potentiellement qqch à faire avec ça!) (extraire du dictionnaire!)
# rating_number (number of ratings for average)


#### Merge Items to Reviews

In [22]:
# Keep only necessary variables before mergeing
df_reviews_f = df_reviews[['rating', 'full_text', 'as_image', 'parent_asin', 'as_helpful_vote', 'helpful_vote', 'verified_purchase']]
df_items_f = df_items_price[['main_category', 'average_rating', 'rating_number', 'price', 'parent_asin', 'categories_grp']]

# Merge Items on Reviews
df_full = df_reviews_f.merge(df_items_f, on='parent_asin', how='left')

# Filter Price
df_full_price = df_full[df_full['price'] != 'None']

# Keep only necessary variables
df_final = df_full_price[['parent_asin', # both
               'rating', 'full_text', 'as_image', 'helpful_vote', 'as_helpful_vote', 'verified_purchase', # reviews
               'main_category', 'average_rating', 'rating_number', 'price', 'categories_grp']] # items

# Filter main_categoy and price
df_final = df_final.dropna()

In [23]:
df_final

,parent_asin,rating,full_text,as_image,helpful_vote,as_helpful_vote,verified_purchase,main_category,average_rating,rating_number,price,categories_grp
0,B002MW50JA,5.0,Title : Five Stars\n Review : LOVE IT!,0,0,0,True,Digital Music,4.6,186.0,19.61,Other
1,B008XNPN0S,5.0,Title : Five Stars\n Review : LOVE!!,0,0,0,True,Digital Music,4.5,49.0,14.38,Other
2,B00IKM5N02,3.0,Title : Three Stars\n Review : Sad there is no...,0,0,0,True,Digital Music,4.6,5324.0,7.76,Other
5,B07Z76Y18X,5.0,Title : Magnificent movie & music score!\n Rev...,0,5,1,True,Digital Music,4.4,276.0,14.98,Other
6,B00004NKAK,5.0,Title : Five Stars\n Review : beautiful music!,0,0,0,True,Digital Music,4.6,80.0,5.99,Other
...,...,...,...,...,...,...,...,...,...,...,...,...
4827268,B000002VPH,5.0,Title : good cd\n Review : I love this cd and ...,0,0,0,True,Digital Music,4.8,1046.0,7.25,Other
4827269,B000084T18,5.0,Title : hot cd\n Review : I love the cd it pla...,0,0,0,True,Digital Music,4.6,2834.0,13.98,Other
4827270,B004OFWLO0,5.0,Title : Superb sounding remaster\n Review : Su...,0,1,1,True,Digital Music,4.8,847.0,15.98,Rock
4827271,B000GIXIAK,1.0,"Title : Very, very disappointing.\n Review : W...",0,6,1,False,Digital Music,4.3,205.0,10.0,Pop


In [14]:
#df_final.iloc[1]['full_text']

### Save


In [24]:
## Save data
df_final.to_csv('./../data/cds_and_vinyl.csv')